In [1]:
using ParallelTemperingMonteCarlo,BenchmarkTools,DelimitedFiles,MachineLearningPotential

Completed the MLP module based on the librunner.so file written by A. Knoll. Time to use this in PTMC simulations so I can finally generate results. This requires the librunnerjulia file and use of the MLP package.

First: establishing the struct.

This requires the symmetry functions, the nnp values itself, the g and f matrices used in the calculation and a dummy vector new_f_vec to preassign memory for later. I'd prefer to point to the nnp and symmetryfunction structs rather than save copies, I'll look into this later. 

In [2]:
struct  RuNNerPotential <: AbstractMachineLearningPotential
    nnp::NeuralNetworkPotential
    symmetryfunctions::Vector{AbstractSymmFunction}
    r_cut::Float64
end
function RuNNerPotential(nnp,symmetryfunction)
    r_cut = symmetryfunction[1].r_cut
    return RuNNerPotential(nnp,symmetryfunction,r_cut)
end


RuNNerPotential

The challenge is potentials are not unique to each given state, meaning that g_matrix, f_matrix need to be part of the mc_state, not necessarily part of an immutable potential. They could be a vector of these matrices but this seems prohibitive. It's possible to assign them to mc_states? Have an NNP subtype?


In [3]:
mutable struct NNPState{T,N,BC}
    temp::T
    beta::T
    config::Config{N,BC,T}
    dist2_mat::Matrix{T}
    en_atom_vec::Vector{T}
    en_tot::T
    ham::Vector{T}
    max_displ::Vector{T}
    count_atom::Vector{Int}
    count_vol::Vector{Int}
    count_rot::Vector{Int}
    count_exc::Vector{Int}

    new_en_atom::Vector{T}

    g_matrix::Array{T}
    f_matrix::Matrix{T}

    new_g_matrix::Array{T}
    new_dist2_vec::Vector{T}
    new_f_vec::Vector{T}
end


In [4]:
function NNPState(state::MCState, positions, n_atoms, symm_function_vector)

    f_matrix = cutoff_function.(sqrt.(state.dist2_mat),Ref(symm_function_vector[1].r_cut))

    g_matrix = total_symm_calc(positions,state.dist2_mat,f_matrix,symm_function_vector)

    nnstate = NNPState(
        state.temp, state.beta, state.config, state.dist2_mat, state.en_atom_vec, state.en_tot, state.ham, state.max_displ, state.count_atom, state.count_vol, state.count_rot, state.count_exc, g_matrix, f_matrix,copy(g_matrix), zeros(n_atoms), zeros(n_atoms)
        )

    state = nnstate
    
    return state
end


NNPState

Functionally, this definition allows us to initialise the runner potential, the state and then use them to define the new potential specific state with the g_matrix included. 

In [5]:
# function energy_calculation(positions,dist2_mat,f_mat,total_symmetry_vector,neuralnetwork)
#     n_atoms = length(positions)
#     g_mat = total_symm_calc(positions,dist2_mat,f_mat,total_symmetry_vector)
#     return g_mat,sum(forward_pass(g_mat,n_atoms,neuralnetwork))
# end
# function threnergy_calculation(positions,dist2_mat,f_mat,total_symmetry_vector,neuralnetwork)
#     n_atoms = length(positions)
#     g_mat = total_thr_symm_calc(positions,dist2_mat,f_mat,total_symmetry_vector)
#     return g_mat,sum(forward_pass(g_mat,n_atoms,neuralnetwork))
# end

# function energy_calculation(g_matrix, positions,newpos,dist2_mat,new_dis_vec,f_mat,new_f_vec,atomindex,total_symmetry_vector,neuralnetwork)
#     n_atoms = length(positions)
#     g_mat = total_symm!(g_matrix, positions,newpos,dist2_mat,new_dis_vec,f_mat,new_f_vec,atomindex,total_symmetry_vector)
#     return g_matrix,sum(forward_pass(g_mat,n_atoms,neuralnetwork))
# end

# function threnergy_calculation(g_matrix, positions,newpos,dist2_mat,new_dis_vec,f_mat,new_f_vec,atomindex,total_symmetry_vector,neuralnetwork)
#     n_atoms = length(positions)
#     g_mat = total_thr_symm!(g_matrix, positions,newpos,dist2_mat,new_dis_vec,f_mat,new_f_vec,atomindex,total_symmetry_vector)
#     return g_matrix,sum(forward_pass(g_mat,n_atoms,neuralnetwork))
# end

In [6]:
function initial_energy_calculation(positions,dist2_mat,f_mat,runner_pot)
    n_atoms = length(positions)
    g_mat = total_symm_calc(positions,dist2_mat,f_mat,runner_pot.symmetryfunctions)
    return g_mat, sum(forward_pass(g_mat,n_atoms,runner_pot.nnp))
end
function initial_energy_calculation(positions,nnp_state,runner_pot)
    n_atoms = length(positions)

    nnp_state.g_matrix = total_symm_calc(positions,nnp_state.dist2_mat,nnp_state.f_matrix,runner_pot.symmetryfunctions)

    nnp_state.en_atom_vec = forward_pass(g_mat,n_atoms,runner_pot.nnp)

    nnp_state.en_tot = sum(nnp_state.en_atom_vec)

    return nnp_state
end

initial_energy_calculation (generic function with 2 methods)

Critically, we require the get_energy function as defined in energyevaluation. Once this is done, we can simulate NNP's. 

Note-- This is done with the following input/output

input:  trial_positions,indices,mc_states,pot

the new positions, the new indices for them, the current states and the potential.

output:  energy_vector, dist2_new




In [7]:
function get_new_state_vars!(trial_pos,atomindex,nnp_state,pot)
    nnp_state.new_dist2_vec = [distance2(pos,b,config.bc) for b in config.pos]
    nnp_state.new_dist2_vec[i_atom] = 0.
    
    nnp_state.new_f_vec = cutoff_function.(sqrt.(dist2_new),Ref(pot.r_cut))


    nnp_state.g_new = copy(nnp_state.g_matrix)

    nnp_state.g_new = total_symm!(g_new,nnp_state.config.pos,trial_pos,nnp_state.dist2_mat,nnp_state.new_dist2_vec,nnp_state.f_matrix,nnp_state.new_f_vec,atomindex,pot.symmetryfunctions)

    return nnp_state
end
function calc_new_energy!(g_new,nnp_state,pot)
    nnp_state.new_en_atom = forward_pass(g_new,length(nnp_state.en_atom_vec),pot.nnp)
    return nnp_state
end

function get_runner_energy!(trial_pos,atomindex,nnp_state,pot)
    nnp_state = get_new_state_vars!(trial_pos,atomindex,nnp_state,pot)
    nnp_state = calc_new_energy!(g_new,nnp_state,pot)

    return sum(nnp_state.new_en_atom),nnp_state
end

function get_energy!(trial_positions,indices,mc_states,pot::RuNNerPotential)
    energyvector,mc_states = get_runner_energy.(trial_positions,indices,mc_states,Ref(pot))
    
    return energyvector,mc_states
end

get_runner_energy! (generic function with 1 method)

Ideally the function above now works the way it's intended in the simulation, other than the natural output being different from other methods. As a result of this we may want to have the g_new_matrix being part of the nnp_state struct, as it preserves the total form of the mc_step function

# Initialisation 

The next major step is to implement a more definitive initialisation. This is somewhat restricted by the purge of .data files, but we can point to directories containing the weights and scaling data. 

Beyond this we need to point to symmetry function values, distributed over sets of atom types. 

In [22]:
#-------------------------------------------#
#--------Vector of radial symm values-------#
#-------------------------------------------#
X = [ 1    1              0.001   0.000  11.338
 1    0              0.001   0.000  11.338
 1    1              0.020   0.000  11.338
 1    0              0.020   0.000  11.338
 1    1              0.035   0.000  11.338
 1    0              0.035   0.000  11.338
 1    1              0.100   0.000  11.338
 1    0              0.100   0.000  11.338
 1    1              0.400   0.000  11.338
 1    0              0.400   0.000  11.338]

radsymmvec = []


#--------------------------------------------#
#--------Vector of angular symm values-------#
#--------------------------------------------#
V = [[0.0001,1,1,11.338],[0.0001,-1,2,11.338],[0.003,-1,1,11.338],[0.003,-1,2,11.338],[0.008,-1,1,11.338],[0.008,-1,2,11.338],[0.008,1,2,11.338],[0.015,1,1,11.338],[0.015,-1,2,11.338],[0.015,-1,4,11.338],[0.015,-1,16,11.338],[0.025,-1,1,11.338],[0.025,1,1,11.338],[0.025,1,2,11.338],[0.025,-1,4,11.338],[0.025,-1,16,11.338],[0.025,1,16,11.338],[0.045,1,1,11.338],[0.045,-1,2,11.338],[0.045,-1,4,11.338],[0.045,1,4,11.338],[0.045,1,16,11.338],[0.08,1,1,11.338],[0.08,-1,2,11.338],[0.08,-1,4,11.338],[0.08,1,4,11.338]]

T = [[1.,1.,1.],[1.,1.,0.],[1.,0.,0.]]

angularsymmvec = []
#-------------------------------------------#
#-----------Including scaling data----------#
#-------------------------------------------#
file = open("$(pwd())/scaling.data")
scalingvalues = readdlm(file)
close(file)
G_value_vec = []
for row in eachrow(scalingvalues[1:88,:])
    max_min = [row[4],row[3]]
    push!(G_value_vec,max_min)
end


for symmindex in eachindex(eachrow(X))
    row = X[symmindex,:]
    radsymm = RadialType2{Float64}(row[3],row[5],[row[1],row[2]],G_value_vec[symmindex])
    push!(radsymmvec,radsymm)
end

n_index = 0
for element in V
    for types in T 
        
        n_index += 1

        symmfunc = AngularType3{Float64}(element[1],element[2],element[3],11.338,types,G_value_vec[n_index])

        push!(angularsymmvec,symmfunc)
    end
end
#---------------------------------------------------#
#------concatenating radial and angular values------#
#---------------------------------------------------#

totalsymmvec = vcat(radsymmvec,angularsymmvec)


#--------------------------------------------------#
#-----------Initialising the nnp weights-----------#
#--------------------------------------------------#
num_nodes::Vector{Int32} = [88, 20, 20, 1]
activation_functions::Vector{Int32} = [1, 2, 2, 1]
file = open("weights.029.data","r+")
weights=readdlm(file)
close(file)
weights = vec(weights)
nnp = NeuralNetworkPotential(num_nodes,activation_functions,weights)

runnerpotential = RuNNerPotential(nnp,totalsymmvec)

RuNNerPotential(NeuralNetworkPotential(4, 2221, Int32[88, 20, 20, 1], Int32[1, 2, 2, 1], [-1.0768943782, 0.3563458393, -1.3084861447, 0.1127640916, -0.5646591931, -0.4969830793, 1.423770514, -0.2005662393, -0.8957859374, 0.6076110858  …  0.1939524848, -0.0342903191, -0.0231042009, 0.0718481968, -0.0828320122, -0.0728501885, 0.0306628826, 0.0486053813, -0.0867018862, 0.0441651841]), AbstractSymmFunction[RadialType2{Float64}(0.001, 11.338, [1.0, 1.0], -0.029892621990594427, 0.06634475788973929), RadialType2{Float64}(0.001, 11.338, [1.0, 0.0], -0.0, 0.09828103647521905), RadialType2{Float64}(0.02, 11.338, [1.0, 1.0], -0.013430592462258787, 0.13321466604629645), RadialType2{Float64}(0.02, 11.338, [1.0, 0.0], -0.0, 0.18618835781674212), RadialType2{Float64}(0.035, 11.338, [1.0, 1.0], -0.0067056953127401205, 0.2093521117141925), RadialType2{Float64}(0.035, 11.338, [1.0, 0.0], -0.0, 0.28571045376567983), RadialType2{Float64}(0.1, 11.338, [1.0, 1.0], -0.00026435144512647774, 0.9364998977117353